In [1]:
from summarizer import Summarizer, TransformerSummarizer
# bert_model = Summarizer()
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")

C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
from openie import StanfordOpenIE

# https://stanfordnlp.github.io/CoreNLP/openie.html#api
# Default value of openie.affinity_probability_cap was 1/3.
properties = {
    'openie.affinity_probability_cap': 2/3,
    # 'openie.resolve_coref': True,
}

In [34]:
text = """
This page shares my best articles to read on topics like health, happiness, creativity, productivity and more. The central question that drives my work is, “How can we live better?” To answer that question, I like to write about science-based ways to solve practical problems.

You’ll find interesting articles to read on topics like how to stop procrastinating as well as personal recommendations like my list of the best books to read and my minimalist travel guide. Ready to dive in? You can use the categories below to browse my best articles.
"""

In [26]:
import nltk
from nltk.corpus import stopwords

def remove_stopwords(text):
    return text.replace('\n', ' ').replace('\r', ' ')

text = remove_stopwords(text)

In [35]:
bert_summary = ''.join(GPT2_model(text, min_length=20))
print(bert_summary)

This page shares my best articles to read on topics like health, happiness, creativity, productivity and more. To answer that question, I like to write about science-based ways to solve practical problems.


In [37]:
text = bert_summary

In [38]:
with StanfordOpenIE(properties=properties) as client:
    annotations = client.annotate(text)

Starting server with command: java -Xmx8G -cp C:\Users\vedan\.stanfordnlp_resources\stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-01e204a019044996.props -preload openie


In [39]:
annotations

[{'subject': 'I', 'relation': 'like', 'object': 'To answer'},
 {'subject': 'I', 'relation': 'answer', 'object': 'question'},
 {'subject': 'I', 'relation': 'like', 'object': 'to write'},
 {'subject': 'I', 'relation': 'like', 'object': 'answer'},
 {'subject': 'I', 'relation': 'like', 'object': 'write'}]

In [31]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [32]:
for i, annotation in enumerate(annotations):
    for j, annotation_2 in enumerate(annotations):
        cmp_string_1 = annotation["subject"] + annotation["relation"] + annotation["object"]
        cmp_string_2 = annotation_2["subject"] + annotation_2["relation"] + annotation_2["object"]
        # print(annotation["subject"], annotation["relation"], annotation["object"])
        # print(annotation_2["subject"], annotation_2["relation"], annotation_2["object"])
        # print(similar(cmp_string_1, cmp_string_2))
        if i!=j and similar(cmp_string_1, cmp_string_2) > 0.6:
            del annotations[j]
            print(annotation["subject"], annotation["relation"], annotation["object"])
            print(annotation_2["subject"], annotation_2["relation"], annotation_2["object"])
            print(similar(cmp_string_1, cmp_string_2))
            print()
    

I like To answer
I like to write
0.6666666666666666

I like answer
I like To answer
0.88

I like answer
I like answer
1.0



In [33]:
annotations

[{'subject': 'I', 'relation': 'answer', 'object': 'question'},
 {'subject': 'I', 'relation': 'like', 'object': 'write'}]

In [42]:
import json

In [47]:
with open("News_Category_Dataset_v2.json", encoding="utf-8") as json_file:
    data = json.load(json_file)

JSONDecodeError: Extra data: line 2 column 1 (char 366)

In [33]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_sm')

C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\spacy\util.py:833: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [29]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [30]:
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1",[pattern]) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

In [31]:
def process(sent):
    return (get_entities(sent), get_relation(sent))

In [36]:
text

'The race to unravel the complex processes occuring on the lunar surface and its environment is heating up. Explaining why it is an important discovery, ISRO said in a statement that Argon-40 (Ar-40) is a noble gas, which makes it one of the most stable gases, and serves as key tracers to understand the processes of surface-exosphere interaction.'

In [39]:
process(text.split(".")[1])

(['key  tracers', 'surface  exosphere'], 'interaction')

In [1]:
import spacy

C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
nlp = spacy.load("./models/spacy_ner_cpu/model-best")

In [5]:
doc = nlp(text)

In [6]:
for ent in doc.ents:
    print(ent.text, ent.label_)

celestial EVENT
radiogenic activities in LOCATION


In [4]:
test = [{'subject': 'I', 'relation': 'answer', 'object': 'question'},
 {'subject': 'I', 'relation': 'like', 'object': 'write'},
 {'subject': 'interesting articles',
  'relation': 'read on',
  'object': 'topics'},
 {'subject': 'You', 'relation': '’ll find', 'object': 'read on topics'},
 {'subject': 'You', 'relation': '’ll find', 'object': 'articles'},
 {'subject': 'You', 'relation': 'browse', 'object': 'my best articles'},
 {'subject': 'You', 'relation': 'can use', 'object': 'categories'}]

In [9]:
def create_nodes(annotations):
    data = {
        "data": {"sub": "summary"},
        "children": []
    }

    parents = []

    for i, annotation in enumerate(annotations):
        if annotation["subject"] not in parents:
            parents.append(annotation["subject"])
            data["children"].append({
                "data": {"sub": annotation["subject"], "click":"click"},
                "children": [
                    {
                        "data": {"sub": annotation["relation"], "ref": "ref"},
                        "children": [{
                            "data": {"sub": annotation["object"]}
                        }]
                    }
                ]
            })
        else:
            index = parents.index(annotation["subject"])
            data["children"][index]["children"].append({
                "data": {"sub": annotation["relation"], "ref": "ref"},
                "children": [{
                    "data": {"sub": annotation["object"]},
                }]
            })

    return data
        

In [10]:
print(create_nodes(test))

{'data': {'sub': 'summary'}, 'children': [{'data': {'sub': 'I', 'click': 'click'}, 'children': [{'data': {'sub': 'answer', 'ref': 'ref'}, 'children': [{'data': {'sub': 'question'}}]}, {'data': {'sub': 'like', 'ref': 'ref'}, 'children': [{'data': {'sub': 'write'}}]}]}, {'data': {'sub': 'interesting articles', 'click': 'click'}, 'children': [{'data': {'sub': 'read on', 'ref': 'ref'}, 'children': [{'data': {'sub': 'topics'}}]}]}, {'data': {'sub': 'You', 'click': 'click'}, 'children': [{'data': {'sub': '’ll find', 'ref': 'ref'}, 'children': [{'data': {'sub': 'read on topics'}}]}, {'data': {'sub': '’ll find', 'ref': 'ref'}, 'children': [{'data': {'sub': 'articles'}}]}, {'data': {'sub': 'browse', 'ref': 'ref'}, 'children': [{'data': {'sub': 'my best articles'}}]}, {'data': {'sub': 'can use', 'ref': 'ref'}, 'children': [{'data': {'sub': 'categories'}}]}]}]}
